# Notebook for performing Mann-Whitney U-tests for statistical significance

### Import stats libraries, namely mannwhitney package from scipy.stats

In [1]:
from statsmodels.sandbox.stats.multicomp import multipletests
from scipy.stats import mannwhitneyu
import xarray as xr
import netCDF4 as nc
from netCDF4 import Dataset
import numpy as np

## Import WRF daily max files (resampled to convective daily max) for each climate epoch

In [3]:
"""
wrf_hist = xr.open_dataset('/home/scratch/jgoodin/convective_daily_max_resamples/hist_daily_max_hail_inches.nc')
wrf_mid4p5 = xr.open_dataset('/home/scratch/jgoodin/convective_daily_max_resamples/mid_century_4p5_daily_max_hail_inches.nc')
wrf_mid8p5 = xr.open_dataset('/home/scratch/jgoodin/convective_daily_max_resamples/mid_century_8p5_daily_max_hail_inches.nc')
wrf_end4p5 = xr.open_dataset('/home/scratch/jgoodin/convective_daily_max_resamples/end_century_4p5_daily_max_hail_inches.nc')
wrf_end8p5 = xr.open_dataset('/home/scratch/jgoodin/convective_daily_max_resamples/end_century_8p5_daily_max_hail_inches.nc')
"""

"\nwrf_hist = xr.open_dataset('/home/scratch/jgoodin/convective_daily_max_resamples/hist_daily_max_hail_inches.nc')\nwrf_mid4p5 = xr.open_dataset('/home/scratch/jgoodin/convective_daily_max_resamples/mid_century_4p5_daily_max_hail_inches.nc')\nwrf_mid8p5 = xr.open_dataset('/home/scratch/jgoodin/convective_daily_max_resamples/mid_century_8p5_daily_max_hail_inches.nc')\nwrf_end4p5 = xr.open_dataset('/home/scratch/jgoodin/convective_daily_max_resamples/end_century_4p5_daily_max_hail_inches.nc')\nwrf_end8p5 = xr.open_dataset('/home/scratch/jgoodin/convective_daily_max_resamples/end_century_8p5_daily_max_hail_inches.nc')\n"

## Pass hail day threshold

In [ ]:
hist_sev = 1 * (wrf_hist >= 0.0254)
mid4p5_sev = 1 * (wrf_mid4p5 >= 0.0254)
mid8p5_sev = 1 * (wrf_mid8p5 >= 0.0254) #Can change threshold to larger hail as well
end4p5_sev = 1 * (wrf_end4p5 >= 0.0254)
end8p5_sev = 1 * (wrf_end8p5 >= 0.0254)

### Resample by year and sum along the 'Time' dimension- provides a count of annual severe hail days for each simulation year. Take mean along 'year' dimension to provide annual sev hail day count

In [ ]:
hist_annual = hist_sev.groupby('Time.year').sum(dim = 'Time').mean(dim = 'year')
mid4p5_annual = mid4p5_sev.groupby('Time.year').sum(dim = 'Time').mean(dim = 'year')
mid8p5_annual = mid8p5_sev.groupby('Time.year').sum(dim = 'Time').mean(dim = 'year') #Groupby year and sum sev days across time dim to provide annual sev hail day count
end4p5_annual = end4p5_sev.groupby('Time.year').sum(dim = 'Time').mean(dim = 'year') #Can change to seasonal as well
end8p5_annual = end8p5_sev.groupby('Time.year').sum(dim = 'Time').mean(dim = 'year')

In [ ]:
hist_annual.shape

### Select array containing HAIL_MAX2D values to send into Mann-Whitney

In [ ]:
hist_annual = hist_annual.HAIL_MAX2D
mid4p5_annual = mid4p5_annual.HAIL_MAX2D
mid8p5_annual = mid8p5_annual.HAIL_MAX2D #Select values (2D array) of HAIL_MAX2D to pass to Mann-Whitney U-test
end4p5_annual = end4p5_annual.HAIL_MAX2D
end8p5_annual = end8p5_annual.HAIL_MAX2D

In [ ]:
hist_annual.shape

## Import netcdf file containing lat/lon coords for WRF files

In [ ]:
coords = xr.open_dataset('lat_lon.nc') #Open .nc file containing WRF lat/lon coords

### Assign lat/lon coords from file to variables

In [ ]:
lats = coords.CLAT.values[0, :, :]
lons = coords.CLONG.values[0, :, :]

## Perform Mann-Whitney U-test for medians (means?), incorporating false-discovery rate correction

In [ ]:
def field_significance(hist_annual, mid4p5_annual):
    
    #sets up p-value array (default 1 - not significant)
    results = np.ones(shape=(hist_annual.shape[1], hist_annual.shape[2]), dtype=float)
    
    #loop through the y and x dimensions
    for i in range(hist_annual.shape[1]):
        for j in range(hist_annual.shape[2]):
            
            dist1 = hist_annual[:, i, j]
            dist2 = mid4p5_annual[:, i, j]
            
            try:
                s, p = mannwhitneyu(dist1, dist2)
                results[i, j] = p
            except Exception as e:
                results[i, j] = np.nan
            
    return results

#Send your arrays into 'field_significance' function, sample below
res = field_significance(hist_annual, mid4p5_annual)
#clip to your region if needed
#res_mask = np.ma.masked_where(clip==False, res).filled(np.nan)
#flat_max = mask_fin[0].values.flatten()

#Perform the false discovery rate test
p = multipletests(res.flatten(), alpha=0.1, method='fdr_bh')[0]

#Save output
#np.save('/home/scratch/jgoodin/fdr_delta_hist_mid4p5_annual_sev_hail_days.npy',p.reshape((lons.shape)))